In [ ]:
import sys
sys.path.append('./../../')

In [1]:
import ikamoana
import parcels
import numpy as np
import xarray as xr
import ikamoana.SEAPODYM_functions_Parcels3 as sfp
import ikamoana
%load_ext line_profiler

## __Initialize Habitat__

In [2]:
xml_filepath="./../../data/feeding_habitat_data/po_2x30_historical_interim/skj_po_interim_2deg_configuration_file.xml"
fh = ikamoana.feedinghabitat.FeedingHabitat(xml_filepath)
ikaf = ikamoana.ikamoanafields.IkamoanaFields(xml_filepath,xml_filepath)

habitat_0_ds = fh.computeFeedingHabitat(0)
habitat_0_ds = habitat_0_ds.sel(time=habitat_0_ds.time[:1])
habitat_0_da = habitat_0_ds['Feeding_Habitat_Cohort_0']

## __Add Ubathy__

In [3]:
ubathy = fh.data_structure.variables_dictionary['forage_lmeso']
habitat_0_ds_to_merge = xr.Dataset(
    {'Ubathy':ubathy},
    coords=habitat_0_ds.coords,
)
habitat_0_ds = habitat_0_ds.merge(habitat_0_ds_to_merge)

## __Convert DataSet to FieldSet__

In [4]:
variables = {
    'H': 'Feeding_Habitat_Cohort_0',
    'Ubathy' : 'Ubathy'}

dimensions = {
    'lon': 'lon',
    'lat': 'lat',
    'time': 'time'}

habitat_0_fs = parcels.FieldSet.from_xarray_dataset(
    habitat_0_ds.reindex(lat=list(reversed(habitat_0_ds.lat))),
    variables=variables,
    dimensions=dimensions)

## __Create Landmask__

In [5]:
landmask_parcels = sfp.Create_Landmask(habitat_0_fs)
landmask_xarray = ikaf.landmask(habitat_0_da)

100% (101 of 101) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [6]:
sum = np.sum(np.flip(landmask_parcels.data[0,:,:], axis=0) == landmask_xarray.data)
compare = landmask_xarray.data.size
assert sum == compare

## __Compute Gradient__

In [7]:
grad_lon_parcels, grad_lat_parcels = sfp.getGradient(
    habitat_0_fs.H, landmask_parcels)

grad_lon_xarray, grad_lat_xarray = ikaf.gradient(
    habitat_0_da, landmask_xarray)

In [8]:
assert (np.sum(grad_lon_parcels.data == grad_lon_xarray.data)
        == grad_lon_xarray.data.size)
assert (np.sum(np.flip(grad_lat_parcels.data, axis=1) == grad_lat_xarray.data)
        == grad_lat_xarray.data.size)

## __Compute Taxis__

In [9]:
taxis_tx, taxis_ty = sfp.Create_SEAPODYM_Taxis_Fields(
    dHdx=grad_lon_parcels, dHdy=grad_lat_parcels,
    length_classes=fh.data_structure.species_dictionary['cohorts_mean_length'],
    start_age=1)

taxis_lon_xarray, taxis_lat_xarray = ikaf.taxis(grad_lon_xarray, grad_lat_xarray)

Taxis for 1 steps


In [24]:
print("D LONGITUDE :")
print("In common :\t",np.sum(taxis_lon_xarray.data == taxis_tx.data))
print("Total : \t",taxis_lon_xarray.size)
print("\nD LATITUDE :")
print("In common :\t",np.sum(taxis_lat_xarray.data == np.flip(taxis_ty.data, axis=1)))
print("Total : \t",taxis_lat_xarray.size)
print("\nSum of Error is :", np.sum(taxis_lon_xarray.data - taxis_tx.data))
print("Corresponding to %.10f %s"%(
    np.sum(taxis_lon_xarray.data-taxis_tx.data)/np.sum(taxis_lon_xarray.data)*100,
    "%")
)

D LONGITUDE :
In common :	 5174
Total : 	 6060

D LATITUDE :
In common :	 6060
Total : 	 6060

Sum of Error is : 2.6788224e-12
Corresponding to 0.0000002471 %
